## Build Chatbot using Google NarrativeQA Reading Comprehension Dataset

### This notebook ingests the updatedSummaryList (cleaned Wikipedia articles to be search for the answer to the question being asked) and the questions_list.  The output is the predicted answer to the question. 

INGEST:
* updatedSummaryList.csv: cleaned Wikipedia articles to tbe searched for the answer to the question being asked 
* questions_list.csv: each question that is being asked 


EXPORT: 
* AlgorithmicAnswers_to_Questions.csv: predicted answers to each question being asked

In [9]:
import os
import pandas as pd
import csv
from os.path import join

In [10]:
import string
import re
import pickle
import nltk
import time

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joelhaas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joelhaas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
startTime = time.time()

In [12]:
# Flatten lists of list 

def nested_flatten(inputList):
    summary = []
    
    for item in inputList:
        if isinstance(item, list):
            summary += nested_flatten(item)
        else:
            summary += [item]
    return summary

In [13]:
####################################
# LIST OF SENTENCES FROM SUMMARIES #
####################################

# read in summary list of each Wikipedia article with each sentence tokenized
with open('updatedSummaryList.csv', newline='') as f:
    spamreader = csv.reader(f, delimiter=',')
    ref_summaries = list(spamreader)
    print(ref_summaries[-5:])

[['523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, just as alice sees police arriving , a decomposing body drags her underwater. '], ['523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, she then awakens in a hospital screaming. '], ['523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, a police officer tells her the aftermath. '], ['523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, when she asks about jason , the officer replies with no evidence of any boy ; alice says he s still there. '], ['523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, the film ends with a peaceful shot of crystal lake. ']]


In [14]:
# flatten list of lists into a single list of sentences of the Wikipedia summaries
ref_summaries = nested_flatten(ref_summaries)
print(ref_summaries[-5:])

['523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, just as alice sees police arriving , a decomposing body drags her underwater. ', '523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, she then awakens in a hospital screaming. ', '523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, a police officer tells her the aftermath. ', '523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, when she asks about jason , the officer replies with no evidence of any boy ; alice says he s still there. ', '523a2eb1ae686d7bf0e664c89d0a490a7e1a22bc, the film ends with a peaceful shot of crystal lake. ']


In [15]:
def cleaning_document(document):
    document = document.replace('"',"")
    return document

In [16]:
#############################
# PREP FUNCTIONS FOR TF-IDF #
#############################

import string

# lemmatize tokens
lemmer = nltk.stem.WordNetLemmatizer()

# remove punctuation
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
translator = str.maketrans("","", string.punctuation)

# lemmatize each token in each segmented sentence
def LemTokens(sent_tokens):
    return [lemmer.lemmatize(token) for token in sent_tokens]

# lemmatize + lower case + remove punctuation
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [17]:
# initialize the TF-IDF function
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

In [18]:
len(ref_summaries)

27696

In [19]:
# create CSV where I will save each predicted answer
with open('AlgorithmicAnswers_to_Questions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["question_id", "question", "answer1_id", "answer1"])

In [20]:
def split_function(response):
    splitResponse = response.split(',')
    response_id = splitResponse[0]

    response = splitResponse[1:]
    response = ''.join(response)
    
    return response_id, response

In [21]:
def response(ref_summaries):
    
    # list to save results to csv
    newRows = []

    # read Questions into a dataframe
    questions = "questions_list.csv"
    df_Q = pd.read_csv(questions)
    
    n=1
    
    # turn the dataframe into a list, where each row is a string in the list
    list_Q_raw = df_Q.values.tolist()
    list_Q = [item for sublist in list_Q_raw for item in sublist]
    
    # iterate through each question 
    for q in list_Q:
        
        try:
            # add the question to the end of the summary sentences
            ref_summaries.append(q)
            #test.append(q)

            ##############################
            # TF-IDF + COSINE SIMILARITY #
            ##############################
            # fit and tranform the data
            tfidf = TfidfVec.fit_transform(ref_summaries)
            #tfidf = TfidfVec.fit_transform(test)

            # perform calculation of cosine similarity between the question added to the end
            #  and all possible answers
            # return an array of scores
            vals = cosine_similarity(tfidf[-1], tfidf)

            # sort the results and return the second to last INDEX of the most similar sentence
            idx = vals.argsort()[0][-2]
            #idx2 = vals.argsort()[0][-3]
            #print('idxs: ', idx, idx2)

            # collapse the array into 1-dimension
            flat = vals.flatten()

            # sort the array by score
            flat.sort()

            # return the cosine similarity SCORE of the most similar sentence
            cosSim_score = flat[-2]
            #cosSim_score2 = flat[-3]
            #print('cosSim_scores: ', cosSim_score, cosSim_score2)

            # Answer: return the top 2 answers that are most similar to the question asked
            answer1_raw = ref_summaries[idx]
            #answer2_raw = ref_summaries[idx2]

            answer1_id = split_function(answer1_raw)[0]
            answer1 = split_function(answer1_raw)[1]

            #answer2_id = split_function(answer2_raw)[0]
            #answer2 = split_function(answer2_raw)[1]

            
            # INDEX of the Question
            idxQ = vals.argsort()[0][-1]
            #print('idxQ: ', idxQ)

            # Return the question
            question_raw = ref_summaries[idxQ]
            #print(n, 'question: ', question)
            #print(n, 'answers: ', answer, answer2)
            question_id = split_function(question_raw)[0]
            question = split_function(question_raw)[1]


            # delete the added question from the end of the list
            del ref_summaries[-1]

            # Save results
            newRows.append((question_id, question, answer1_id, answer1))

            print(n)
            n+=1
        except:
            print('error')

    # save predicted answers to csv
    responseResults = open('AlgorithmicAnswers_to_Questions.csv', 'a')
    with responseResults:
        writer = csv.writer(responseResults)
        writer.writerows(newRows)
    

In [22]:
response(ref_summaries)

/Users/joelhaas/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Users/joelhaas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


1


/Users/joelhaas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


2


/Users/joelhaas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


...

/Users/joelhaas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


1038


/Users/joelhaas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


1039
1040


/Users/joelhaas/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


In [23]:
endTime = time.time()
totalTime = (endTime - startTime)/60
print(str(totalTime) + 'min')

115.18317998250326min
